<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Recurrent Neural Network (RNN)**

Here, we decided to employ a Recurrent Neural Network (RNN) using Keras, specifically utilizing an LSTM (Long Short-Term Memory) architecture, to classify text difficulty levels. The RNN was chosen for its ability to understand sequential data, making it well-suited for text. We first tokenized and padded our text data, then used an Embedding layer to create dense, meaningful representations of words. 

The LSTM layer, known for handling long-term dependencies in text, was combined with Dense layers for classification. Despite its sophistication, this approach required careful tuning and ample training data to effectively capture the nuances of text complexity. The final model demonstrated promising accuracy in identifying different levels of text difficulty.

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
df_train = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/training_data.csv").dropna()
df_test = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/unlabelled_test_data.csv").dropna()
df_final = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/sample_submission.csv").dropna()

In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

training_data = df_train
unlabelled_test_data = df_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_data['sentence'])
X = tokenizer.texts_to_sequences(training_data['sentence'])
X = pad_sequences(X)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(training_data['difficulty'])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

X_unlabelled = tokenizer.texts_to_sequences(unlabelled_test_data['sentence'])
X_unlabelled = pad_sequences(X_unlabelled, maxlen=X.shape[1])

predicted_levels = model.predict(X_unlabelled)
predicted_labels = label_encoder.inverse_transform([np.argmax(p) for p in predicted_levels])

unlabelled_test_data['predicted_difficulty'] = predicted_labels
print(unlabelled_test_data)
